In [9]:
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import gbq
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'

In [10]:
bigquery_client = bigquery.Client()
query = "SELECT * FROM `lab1-449822.walmartsales.sales`"
query_result = bigquery_client.query(query)
df_sales = query_result.to_dataframe()

In [11]:
df_sales.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,SalesDate,TransactionNumber
0,5620027,1,434,1,1,10.0,2018-03-14 16:47:01.010000+00:00,TV0JN49XI481EBPBX0BJ
1,1874651,16,1279,1,1,10.0,2018-04-04 10:42:11.560000+00:00,CK2ZIPN1IYVCBENKIULI
2,4652836,9,2427,1,1,0.0,2018-01-25 09:00:44.700000+00:00,TQJXF2UFMZ7COXZPPZD1
3,1839502,8,62,2,1,5.0,2018-01-04 13:45:15.410000+00:00,2G2IX0YMX8IZKC7FYS8F
4,4121154,1,1611,2,1,10.0,2018-04-10 09:51:24.140000+00:00,LLMT1UOUDTL08H4BJXPZ


In [12]:
df_sales['SalesDate'] = pd.to_datetime(df_sales['SalesDate']).dt.date
df_sales.head()


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,SalesDate,TransactionNumber
0,5620027,1,434,1,1,10.0,2018-03-14,TV0JN49XI481EBPBX0BJ
1,1874651,16,1279,1,1,10.0,2018-04-04,CK2ZIPN1IYVCBENKIULI
2,4652836,9,2427,1,1,0.0,2018-01-25,TQJXF2UFMZ7COXZPPZD1
3,1839502,8,62,2,1,5.0,2018-01-04,2G2IX0YMX8IZKC7FYS8F
4,4121154,1,1611,2,1,10.0,2018-04-10,LLMT1UOUDTL08H4BJXPZ


In [14]:
try:
    df_sales.to_gbq(
    destination_table='walmartsales.sales_final', 
    project_id='lab1-449822',            
    if_exists='fail',)
    
except gbq.TableCreationError as e:
    if "Already Exist" in str(e):
        print("The Table Already Exists")
    else:
        raise
    


C:\Users\apoor\AppData\Local\Temp\ipykernel_7812\1341140065.py:2: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_sales.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 6472.69it/s]
